In [0]:
import pandas as pd
import numpy as np
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')
request = drive_service.files().get_media(fileId = '1XQgPGGrzOU24SGcujJOZ9mPpDDfjaV4o').execute()
import io
data = pd.read_csv(io.BytesIO(request))
data.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,N712SW,128.0,150.0,116.0,-14.0,8.0,IAD,TPA,810,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,N772SW,128.0,145.0,113.0,2.0,19.0,IAD,TPA,810,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,2008,1,3,4,628.0,620,804.0,750,WN,448,N428WN,96.0,90.0,76.0,14.0,8.0,IND,BWI,515,3.0,17.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,2008,1,3,4,926.0,930,1054.0,1100,WN,1746,N612SW,88.0,90.0,78.0,-6.0,-4.0,IND,BWI,515,3.0,7.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,3920,N464WN,90.0,90.0,77.0,34.0,34.0,IND,BWI,515,3.0,10.0,0,NaN,0,2.0,0.0,0.0,0.0,32.0


In [0]:
# Handling missing values
data = data.drop(columns=['CancellationCode', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay', 'Year'])
data = data.dropna(subset=['DepTime', 'ArrTime', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
                          'DepDelay', 'TaxiIn', 'TaxiOut'])
data = data.drop(columns=['TailNum', 'Origin', 'Dest','DepDelay', 'Cancelled','Diverted'])
# Tạo cột mới với encode cho target column ArrDelay
data['ArrDelay_ecd'] = np.where(data['ArrDelay'] >= 30, 1, 0)
# xóa cột ArrDelay đi vì ta đã có cột ArrDelay_ecd
data = data.drop(columns=['ArrDelay'])
# check missing value again
missing_val_in_column = (data.isnull().sum())
print(missing_val_in_column[missing_val_in_column >0])

Series([], dtype: int64)


In [0]:
# Down Data
data_down = data.sample(frac=0.35) # lấy mẫu ngẫu nhiên xuống còn 35%
del data

In [0]:
# remove giá trị bất thường CRSElapsedTime
data_down = data_down.drop(data_down[data_down['CRSElapsedTime'] < 0].index, axis=0)

In [0]:
data_down.shape

(2399253, 16)

In [0]:
data_down['ArrDelay_ecd'].value_counts()

0    2075375
1     323878
Name: ArrDelay_ecd, dtype: int64

### Apply PCA với n_components = 7

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components= 7)

In [0]:
X = data_down.drop(['ArrDelay_ecd', 'Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'], axis=1)
y = data_down["ArrDelay_ecd"] # target column

In [0]:
# Encode cái feature category
n= data_down[['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier']]
one_hot = pd.get_dummies(data=n, columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier'])

In [0]:
X = pca.fit_transform(X) # PCA các feature không phải category
X = pd.DataFrame(X) # chuyển về data frame

In [0]:
X = X.reset_index()
one_hot = one_hot.reset_index()
X = X.drop('index', axis=1) # drop cột index tạo ra khi reset_index
one_hot = one_hot.drop('index', axis=1) # # drop cột index tạo ra khi reset_index
new_X = pd.concat([X, one_hot], axis=1, sort=False) # Input của chúng ta sau khi đã xử lý

In [0]:
new_X.shape

(2399253, 77)

**Hold out**

In [0]:
x_train, x_val, y_train, y_val = train_test_split(new_X, y, test_size=0.2, random_state=0)

### Grid Search CV

In [0]:
from sklearn.model_selection import GridSearchCV

**1.1 Model Logistic Regression dùng mặc định chưa tune**

In [0]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.92      1.00      0.96    414976
           1       0.96      0.45      0.62     64875

    accuracy                           0.92    479851
   macro avg       0.94      0.72      0.79    479851
weighted avg       0.93      0.92      0.91    479851



**1.2 Model Logistic Regression thử bộ tham số trên GridSearchCV**

In [0]:
from sklearn.linear_model import LogisticRegression
param_grid = {'penalty': ['l2'],
             'C' : [0.7,0.8,0.9],
             'solver': ['lbfgs', 'liblinear'],
             'class_weight': [{1:0.6, 0:0.4}, {1:0.7, 0:0.3}, {1:0.8, 0:0.2}]}
logreg_grid = GridSearchCV(estimator=LogisticRegression(),
                          param_grid = param_grid,
                          scoring="f1",
                          cv=3,
                          n_jobs = 1)
logreg_grid.fit(x_train, y_train)
logreg_grid_best = logreg_grid.best_estimator_ #best estimator
print("Best Model Parameter: ",logreg_grid.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/usr/local

Best Model Parameter:  {'C': 0.8, 'class_weight': {1: 0.7, 0: 0.3}, 'penalty': 'l2', 'solver': 'lbfgs'}


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [0]:
logreg_grid_best

LogisticRegression(C=0.8, class_weight={0: 0.3, 1: 0.7}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
print("Best Model Parameter: ",logreg_grid.best_params_)

Best Model Parameter:  {'C': 0.8, 'class_weight': {1: 0.7, 0: 0.3}, 'penalty': 'l2', 'solver': 'lbfgs'}


In [0]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = logreg_grid_best
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_val)
print(classification_report(y_val, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.93      0.98      0.96    415316
           1       0.85      0.54      0.66     64535

    accuracy                           0.93    479851
   macro avg       0.89      0.76      0.81    479851
weighted avg       0.92      0.93      0.92    479851



Sau khi tune xong thì kết quả được cải thiện, f1-score của label 1 đã tăng lên 0.66 so với mặc định 0.62.

## Conclusions of this section
- Việc tunning trên model tốn nhiều thời gian, việc thử bộ thông số càng nhiều thì thời gian tăng càng cao.<br>

Vậy các bước nên làm ở đây là gì? Ý tưởng:
- Tìm ra mô hình tốt nhất bằng cách thử một loạt các mô hình trên PCA và SelectKbest.
- Khi chọn ra mô hình tốt nhất rồi thì RFE để nó có thể chọn feature tốt nhất cho mô hình đó.
- Dùng GridSearchCV để tunning mô hình bằng bộ tham số. Chạy lại mô hình bằng bộ thông số đó để thử trên tập **test** bởi GridSearchCV hoàn toàn là thử trên tập **train**.

Một số câu hỏi phụ: liệu dùng label encoder trên các feature category rồi PCA hoặc SelectKbest hết các tập X thì kết quả liệu có tốt hơn? Thời gian train liệu có nhanh hơn?